# GPU-Jupyter

This Jupyterlab Instance is connected to the GPU via CUDA drivers. In this notebook, we test the installation and perform some basic operations on the GPU.

## Test GPU connection

#### Using the following command, your GPU type and its NVIDIA-SMI driver version should be listed:

In [3]:
!nvidia-smi

Wed Feb 22 17:27:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         Off  | 00000000:00:1E.0 Off |                    0 |
|  0%   15C    P8    22W / 300W |      0MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Now, test if PyTorch can access the GPU via CUDA:

In [4]:
import torch
torch.cuda.is_available()

True

In [5]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(tf.config.list_physical_devices('XLA_GPU'))
device_lib.list_local_devices()

2023-02-22 17:27:42.679193: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-22 17:27:43.431960: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-22 17:27:43.432095: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-22 17:27:43.432105: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

[]


2023-02-22 17:27:44.225243: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-22 17:27:44.227183: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-02-22 17:27:44.227262: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2023-02-22 17:27:44.227334: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2023-02-22 17:27:44.227403: W tensorflow/compile

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18011629764085668391
 xla_global_id: -1]

device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
from __future__ import print_function
import numpy as np
import torch
a = torch.rand(5, 3)
a

tensor([[0.5439, 0.1642, 0.7159],
        [0.7129, 0.5600, 0.0823],
        [0.3805, 0.8895, 0.4125],
        [0.3480, 0.7195, 0.7120],
        [0.3308, 0.9598, 0.4042]])

## Performance test

#### Now we want to know how much faster a typical operation is using GPU. Therefore we do the same operation in numpy, PyTorch and PyTorch with CUDA. The test operation is the calculation of the prediction matrix that is done in a linear regression.

### 1) Numpy

In [7]:
x = np.random.rand(10000, 256)

In [8]:
%%timeit
H = x.dot(np.linalg.inv(x.transpose().dot(x))).dot(x.transpose())

229 ms ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 2) PyTorch

In [9]:
x = torch.rand(10000, 256)

In [10]:
%%timeit
# Calculate the projection matrix of x on the CPU
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

174 ms ± 30.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 3) PyTorch on GPU via CUDA

In [11]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 256, device=device) # directly create a tensor on GPU
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])
    print(y.to("cpu", torch.double)[0:5, 0:5])

tensor([[0.1250, 0.5978, 0.2629, 0.4280, 0.0699],
        [0.2525, 0.3196, 0.4418, 0.9479, 0.3076],
        [0.1173, 0.8907, 0.2445, 0.3802, 0.9915],
        [0.3198, 0.9632, 0.2504, 0.1071, 0.6853],
        [0.3151, 0.7957, 0.0842, 0.2706, 0.7991]], device='cuda:0')
tensor([[0.1250, 0.5978, 0.2629, 0.4280, 0.0699],
        [0.2525, 0.3196, 0.4418, 0.9479, 0.3076],
        [0.1173, 0.8907, 0.2445, 0.3802, 0.9915],
        [0.3198, 0.9632, 0.2504, 0.1071, 0.6853],
        [0.3151, 0.7957, 0.0842, 0.2706, 0.7991]], dtype=torch.float64)


In [12]:
%%timeit
# Calculate the projection matrix of x on the GPU
H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

3.76 ms ± 30.3 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Exhaustive Testing on GPU

In [13]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    x = torch.rand(10000, 10, device=device) # directly create a tensor on GPU

In [14]:
if torch.cuda.is_available():
    y = x.to(device)                       # or just use strings ``.to("cuda")``
    print(x[0:5, 0:5])

tensor([[0.4952, 0.1978, 0.3260, 0.1403, 0.1697],
        [0.1066, 0.0579, 0.1995, 0.3621, 0.5795],
        [0.4625, 0.3633, 0.9336, 0.4001, 0.0506],
        [0.7668, 0.4487, 0.0754, 0.9360, 0.1379],
        [0.2859, 0.9974, 0.8614, 0.2845, 0.7079]], device='cuda:0')


In [15]:
if torch.cuda.is_available():
    # Here is the memory of the GPU a border. 
    # A matrix with 100000 lines requires 37 GB, but only 8 GB are available.
    H = x.mm( (x.t().mm(x)).inverse() ).mm(x.t())

In [16]:
if torch.cuda.is_available():
    print(H[0:5, 0:5])

tensor([[ 6.4966e-04,  2.9388e-04,  2.6919e-04, -2.1088e-04, -2.7866e-04],
        [ 2.9388e-04,  1.0254e-03,  2.8319e-04,  2.2687e-04,  7.5790e-06],
        [ 2.6919e-04,  2.8319e-04,  1.4405e-03,  1.6461e-04,  2.0644e-04],
        [-2.1088e-04,  2.2687e-04,  1.6461e-04,  1.4075e-03, -6.9897e-05],
        [-2.7866e-04,  7.5791e-06,  2.0644e-04, -6.9897e-05,  9.9871e-04]],
       device='cuda:0')


In [17]:
if torch.cuda.is_available():
    # This operation is difficult, as an symmetric matrix is transferred 
    # back to the CPU. Is possible up to 30000 rows.
    print(H.to("cpu", torch.double)[0:5, 0:5])

tensor([[ 6.4966e-04,  2.9388e-04,  2.6919e-04, -2.1088e-04, -2.7866e-04],
        [ 2.9388e-04,  1.0254e-03,  2.8319e-04,  2.2687e-04,  7.5790e-06],
        [ 2.6919e-04,  2.8319e-04,  1.4405e-03,  1.6461e-04,  2.0644e-04],
        [-2.1088e-04,  2.2687e-04,  1.6461e-04,  1.4075e-03, -6.9897e-05],
        [-2.7866e-04,  7.5791e-06,  2.0644e-04, -6.9897e-05,  9.9871e-04]],
       dtype=torch.float64)
